# Graph

Tensorflow 1.x uses static graph, which means TF needs to create the whole compute graph before starting a seesion for the calculation.

Tensorflow 2.x uses dynamic graph as defect setting, which means that TF adds new operator automaticly to the graph and needs no session for the calculation. It's also called eager mode. Dynamic graph is friendly to debugging, but can be low efficient. Because dynamic graph needs several communications between python process and Tensorflow C++ process, while the static graph can be excuted on the C++ core code. 

In order to be both friendly to debugging and effcient, Tensorflow gives the autograph by the decorator @tf.function.

In [20]:
import tensorflow as tf
import numpy as np

## Static graph

In [10]:
g = tf.compat.v1.Graph()
with g.as_default():
    x = tf.compat.v1.placeholder(name='x', shape=[], dtype=tf.string)
    y = tf.compat.v1.placeholder(name='y', shape=[], dtype=tf.string)
    z = tf.strings.join([x,y],name = "join",separator = " ")
    
with tf.compat.v1.Session(graph=g) as sess:
    result = sess.run(fetches=z, feed_dict={x: "hello", y:"world"})
tf.print(result)

b'hello world'


## Dynamic graph

In [8]:
x = tf.constant("hello")
y = tf.constant("world")
z = tf.strings.join([x,y],separator=" ")

tf.print(z)

hello world


In [9]:
## define a function to realize str join
def strjoin(x, y):
    z = tf.strings.join([x, y], separator=" ")
    tf.print(z)
    
    return z

res = strjoin(tf.constant("hi"), tf.constant("tf2"))
tf.print(res)

hi tf2
hi tf2


## Autograph

In [11]:
@tf.function
def strjoin(x, y):
    z = tf.strings.join([x, y], separator=" ")
    tf.print(z)
    
    return z

In [12]:
result = strjoin(tf.constant("hello"),tf.constant("world"))

hello world


In [14]:
tf.print(result)

hello world


In [15]:
import datetime

stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = './data/autograph/%s' % stamp
writer = tf.summary.create_file_writer(logdir)


tf.summary.trace_on(graph=True, profiler=True) 

result = strjoin("hello","world")

with writer.as_default():
    tf.summary.trace_export(
        name="autograph",
        step=0,
        profiler_outdir=logdir)

hello world


## Suggestions for autograph

1. Use tf functions in the function needs decorator @tf.function. For example, replace print by tf.print, replace range by tf.range, etc.
2. Don't define tf.Variable in the function
3. The function decorated by @tf.function can't modify the external variables.

In [18]:
@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)

@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [21]:
## it's not random with numpy 
np_random()
np_random()

array([[-0.931798  , -1.18049386,  0.32316736],
       [-1.71565975, -0.09799514, -0.67729762],
       [ 0.01863867, -0.517562  , -0.29465478]])
array([[-0.931798  , -1.18049386,  0.32316736],
       [-1.71565975, -0.09799514, -0.67729762],
       [ 0.01863867, -0.517562  , -0.29465478]])


In [23]:
## random with tf function
tf_random()
tf_random()

[[0.90325278 0.616142452 -0.399411827]
 [0.247912437 0.622742951 -0.211246192]
 [-0.352653742 -0.134327292 -0.0862187073]]
[[0.445511788 0.550857604 -1.23167014]
 [-0.283226758 0.551464379 0.611933887]
 [0.650778115 1.35756159 0.0196729377]]


In [24]:
x = tf.Variable(1.0,dtype=tf.float32)
@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return(x)

outer_var() 
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [25]:
@tf.function
def inner_var():
    x = tf.Variable(1.0,dtype = tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return(x)

In [27]:
## error, if you define tf.Variable in the @tf.function function
inner_var()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


ValueError: in converted code:

    <ipython-input-25-794498ecfe19>:3 inner_var  *
        x = tf.Variable(1.0,dtype = tf.float32)
    /Users/ShengFANG/.virtualenvs/kaggle/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /Users/ShengFANG/.virtualenvs/kaggle/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /Users/ShengFANG/.virtualenvs/kaggle/lib/python3.6/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /Users/ShengFANG/.virtualenvs/kaggle/lib/python3.6/site-packages/tensorflow_core/python/eager/def_function.py:502 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


In [28]:
## change tensor_list 
tensor_list = []

def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [29]:
## fail to change tensor_list
tensor_list = []

@tf.function
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list


append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)

[<tf.Tensor 'x:0' shape=() dtype=float32>]
